#Policy-Gradient and Actor-Critic 

Landing on the moon environement

In [ ]:
#https://stackoverflow.com/questions/64161280/rl-problem-on-colab-for-gym-envs-box2d-has-no-attribute-lunarlander
!pip3 install box2d-py
!pip3 install gym[Box_2D]
import gym
env = gym.make("LunarLander-v2")

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [ ]:
class ActorCriticNetwork(nn.Module):
  def __init__(self, input_dims, n_actions, lr=0.005):
    super(ActorCriticNetwork, self).__init__()
    self.fc1 = nn.Linear(*input_dims, 1536)
    self.fc2 = nn.Linear(1536, 2048)
    self.actor = nn.Linear(2048, n_actions)
    self.critic = nn.Linear(2048, 1)
    
    self.optimizer = optim.Adam(self.parameters(), lr=lr)
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.to(self.device)


  def forward(self, state):
    state = np.expand_dims(state, axis=0)
    if not torch.is_tensor(state):
      state = torch.tensor(state).float().to(self.device)  
    out = F.relu(self.fc1(state))
    out = F.relu(self.fc2(out))
    
    #mine
    probs = F.softmax(self.actor(out), dim=1) #more numericallly stably than softmax and then log is F.log_softmax
    values = self.critic(out.detach())
    return probs, values


#sanity checks
#mod = ActorCriticNetwork(env.reset().shape, 4)

#obs_batch = torch.randn(1, 8)

#state_actions_probs = mod.forward(obs_batch)
#state_actions_probs


(tensor([[[1., 1., 1., 1.]]], grad_fn=<SoftmaxBackward0>),
 tensor([[[-0.0147]]], grad_fn=<AddBackward0>))

In [ ]:
class ActorCriticAgent():

  def __init__(self, input_dims, n_actions, lr=0.005, gamma=0.99):
    self.gamma = gamma
    self.lr = lr
    self.network = ActorCriticNetwork(input_dims, n_actions, lr=self.lr )
  

  def choose_action(self, obs):
    #this does NOT produce probabilities -see note up above why I got this wrong the 1st time
    probs, _ = self.network.forward(obs)
    action_probs = torch.distributions.Categorical(probs)
    action = action_probs.sample()
    log_prob = action_probs.log_prob(action)
    return (action.item(), log_prob )  # m.log_prob(action))


  def learn(self, state, reward, next_state, done, log_prob):

    self.network.zero_grad()

    delta = reward + self.gamma * self.network.forward(next_state)[1] * (1-int(done)) - self.network.forward(state)[1]

    ACTOR_loss = -1* log_prob * delta
    critic_loss = delta**2
    loss = ACTOR_loss + critic_loss
    loss.backward()
    self.network.optimizer.step()








In [ ]:
#main loop 
n_eps = 2000
scores = []
agent = ActorCriticAgent(env.reset().shape, env.action_space.n, lr= 5e-6)
for eps in range(n_eps):
  done, state, score = False, env.reset(), 0
  while not done:
    action, log_prob  = agent.choose_action(state)
    next_state, reward, done, _ = env.step(action)
    agent.learn(state, reward, next_state, done, log_prob)
    score += reward
    state = next_state
  scores.append(score)
  avg_score = np.mean(scores[-100:])
  print("episode: ", eps, 
            "score:", score, 
            " average score %.1f" % avg_score)
  print(eps)

episode:  0 score: -109.23370925007633  average score -109.2
0
episode:  1 score: -378.18515971123736  average score -243.7
1
episode:  2 score: -43.693016224532954  average score -177.0
2
episode:  3 score: -117.0568510232946  average score -162.0
3
episode:  4 score: -101.18211627683256  average score -149.9
4
episode:  5 score: -97.59807371495143  average score -141.2
5
episode:  6 score: -94.25397202815708  average score -134.5
6
episode:  7 score: -372.28421940124684  average score -164.2
7
episode:  8 score: -64.02389549539345  average score -153.1
8
episode:  9 score: -94.09973712864128  average score -147.2
9
episode:  10 score: -112.85972612300424  average score -144.0
10
episode:  11 score: -365.83565736911703  average score -162.5
11
episode:  12 score: -101.65034884627522  average score -157.8
12
episode:  13 score: -340.74168541336695  average score -170.9
13
episode:  14 score: -81.70919922460088  average score -165.0
14
episode:  15 score: -62.30917678210759  average sco

KeyboardInterrupt: ignored